* undiscounted episodic tasks
* 2 actions from each state, each resulting in one of $b$ states, all equally likely
* different random selection of $b$ states for each state-action pair
* for each transition theres a $0.1$ chance of entering a terminal state
* expected reward for each transition sampled from $\mathcal{N}(0,1)$

In [2]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt

In [3]:
class Environment:
    def __init__(self, n, b):
        self.states = n
        self.branch = b
        
        self.build_transition_graph()
        
    def build_transition_graph(self):
        '''build neighbors for each state randomly'''
        self.trans  = np.zeros((self.states, self.branch), dtype=int)
        
        for n in range(self.states):
            for b in range(self.branch):
                self.trans[n, b] = np.random.randint(n)
                
    def is_terminal(self):
        '''Call to determine if move is terminal'''
        return np.random.rand() < 0.1
    
    def get_reward(self):
        '''Returns reward as defined'''
        return np.random.normal()